In [1]:
# Dieses Skript lädt das vortrainierte Modell und überprüft die Inferenz-Geschwindigkeit mittels der Intel Extension for Pytorch.
# Basiert auf dem Detectron2 Tutorial: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5
#
# This code is available under a GPL v3.0 license and comes without
# any explicit or implicit warranty.
#
# (C) Simon Schwaiger 2024 <schwaige@technikum-wien.at>


import os
import cv2


## Import von Detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Laden des Datensatzes
datasetPath = "/app/detectronDataset"
classesFile = os.path.join(datasetPath, "classNames.txt")
modelWeightFile = os.path.join(datasetPath, "model_final.pth")

def getFiles(directory, ext=(".jpg", ".png")):
    """Gibt alle Dateien in einem Verzeichnis von bestimmten Dateitypen zurück """
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if ext == []: retArr = files
    else: retArr = [ file for file in files if file.endswith(ext) ]
    return retArr

trainImages = getFiles(datasetPath)

# Lesen der Klassennamen
with open(classesFile, "r") as f:
    classesList = f.read().splitlines()

print("Found {} dataset images (without augmentation)".format(len(trainImages)))
print("Read the following classes:")
print(classesList)

def formatCOCODict(idx, imgname, datasetPath=datasetPath):
    """Formatiert ein Bildt im COCO Format (Bild + Annotierung) zurück """
    # Pfad von Bild und Bildbeschreibung
    imgpath = os.path.join(datasetPath, imgname)
    annotationPath = os.path.splitext(imgpath)[0] + ".txt"
    # Ermittlung der Bildbreite und Höhe
    img = cv2.imread(imgpath)
    width = img.shape[1]
    height = img.shape[0]
    # Lesen der Bildannotierung
    with open(annotationPath, "r") as f:
        annotationLines = f.readlines()
    # Konvertierung der Annotierung und Speichern der Metadaten
    annotations = []
    for line in annotationLines:
        classIdx, x, y, w, h = line.split(" ")
        annotations.append(
            {
                "bbox": [float(x)*width-(float(w)*width/2), float(y)*height-(float(h)*height/2), float(w)*width, float(h)*height],
                "bbox_mode": 1,
                "category_id": int(classIdx)
            }
        )
    #
    return {
        "file_name": imgpath,
        "image_id": idx,
        "width": width,
        "height": height,
        "annotations": annotations
    }

# Lediglich das erste Bild im Datensatz wird als Testbild verwendet, da wir nur weniger Bilder zum Trainieren zur Verfügung haben
def getMarkerDicts(trainImages=trainImages[1:]):
    return [
        formatCOCODict(idx, imgname)
        for idx, imgname in enumerate(trainImages)
    ]

def getMarkerDictsTest(trainImages=trainImages[:1]):
    return [
        formatCOCODict(idx, imgname)
        for idx, imgname in enumerate(trainImages)
    ]

# Registrierung der Datensätze
DatasetCatalog.register("marker_train", getMarkerDicts)
MetadataCatalog.get("marker_train").set(thing_classes=classesList)
marker_metadata = MetadataCatalog.get("marker_train")

## Erstellung einer Modellkonfiguration, die dem trainierten Modell gleicht
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("marker_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classesList)

# Laden des vortrainierten Modells
cfg.MODEL.WEIGHTS = modelWeightFile
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.MODEL.DEVICE = 'cpu'

# Instanziierung des Predictors basierend auf der Konfiguration
predictor = DefaultPredictor(cfg)

## Laden der Bildvisualisierung
from detectron2.utils.visualizer import ColorMode


Found 23 dataset images (without augmentation)
Read the following classes:
['WinkelGross', 'Sicherung', 'Zahnrad', 'LWinkel']


In [2]:
## Optimierung des Modells mittels intel extension for pytorch, um Inferenz auf eingebetteter Intel CPU zu beschleunigen
import intel_extension_for_pytorch as ipex

model = ipex.optimize(predictor.model)
predictor.model = model

/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:462: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:469: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


In [4]:
%%time
## Messung der Zeit zur Voraussage aller Bilder im Datensatz

# Anzeige des Fortschritts
from ipywidgets import IntProgress
from IPython.display import display

f = IntProgress(min=0, max=len(trainImages))
display(f)

for imgname in trainImages:
    imgpath = os.path.join(datasetPath, imgname)
    img = cv2.imread(imgpath)
    outputs = predictor(img)
    f.value += 1

IntProgress(value=0, max=23)

/opt/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


CPU times: user 1min 46s, sys: 7.88 s, total: 1min 53s
Wall time: 28.7 s
